In [3]:
import sys

from challenge import agoda_cancellation_estimator

sys.path.append("../")
from utils import *
from agoda_cancellation_estimator import *
from agoda_cancellation_prediction import *
from agoda_cancellation_preprocessor import AgodaCancellationPreprocessor
from sklearn.metrics import roc_curve, auc, f1_score
from IMLearn.utils import split_train_test

np.random.seed(0)
c = [custom[0], custom[-1]]

# Load and preprocess data
full_data = load_data("../datasets/agoda_cancellation_train.csv")
p = AgodaCancellationPreprocessor(full_data)
design_matrix = p.preprocess(full_data)
cancellation_labels = p.preprocess_labels(full_data.cancellation_datetime,
                                          full_data.booking_datetime)
test_set = p.preprocess(pd.read_csv("test_set_week_1.csv"))
test_set_labels = pd.read_csv("test_set_week_1_labels.csv")["h_booking_id|label"].astype(str).apply(lambda x: int(x[-1]))
missing_cols = set(design_matrix.columns) - set(test_set.columns)
for c in missing_cols:
    test_set[c] = 0
test_set = test_set[design_matrix.columns]

In [4]:
import agoda_cancellation_estimator
import importlib
importlib.reload(agoda_cancellation_estimator)
from agoda_cancellation_estimator import AgodaCancellationEstimator

train_X = design_matrix
train_y = cancellation_labels
test_X = test_set
test_y = test_set_labels


# Fit model over data
C=1
for epsilon in range(180,210):
    model = AgodaCancellationEstimator(C, epsilon/1000, 0.33).fit(train_X, train_y)
    y_prob = model.predict(test_X)
    f1_macro = f1_score(test_y, y_prob, labels=[0,1], average="macro")
    print(f"C: {C}, epsilon: {epsilon/1000}, f1 macro: {f1_macro}")


# fpr, tpr, thresholds = roc_curve(test_y, y_prob)

# go.Figure(
#     data=[go.Scatter(x=[0, 1], y=[0, 1], mode="lines",
#                      line=dict(color="black", dash='dash'),
#                      name="Random Class Assignment"),
#           go.Scatter(x=fpr, y=tpr, mode='markers+lines', text=thresholds,
#                      name="", showlegend=False, marker_size=5,
#                      # marker_color=c[1][1],
#                      hovertemplate="<b>Threshold:</b>%{text:.3f}<br>FPR: %{x:.3f}<br>TPR: %{y:.3f}")],
#     layout=go.Layout(
#         title=rf"$\text{{ROC Curve Of Fitted Model - AUC}}={auc(fpr, tpr):.6f}$",
#         xaxis=dict(title=r"$\text{False Positive Rate (FPR)}$"),
#         yaxis=dict(title=r"$\text{True Positive Rate (TPR)}$")))

C: 1, epsilon: 0.18, f1 macro: 0.5202850877192983
C: 1, epsilon: 0.181, f1 macro: 0.5202850877192983
C: 1, epsilon: 0.182, f1 macro: 0.5202850877192983
C: 1, epsilon: 0.183, f1 macro: 0.5202850877192983
C: 1, epsilon: 0.184, f1 macro: 0.5202850877192983
C: 1, epsilon: 0.185, f1 macro: 0.5202850877192983
C: 1, epsilon: 0.186, f1 macro: 0.5202850877192983
C: 1, epsilon: 0.187, f1 macro: 0.518964332424464
C: 1, epsilon: 0.188, f1 macro: 0.518964332424464
C: 1, epsilon: 0.189, f1 macro: 0.5176987339591766


KeyboardInterrupt: 

In [12]:
model = AgodaCancellationEstimator().fit(design_matrix, cancellation_labels)
y_prob = model.predict(design_matrix)
fig = px.histogram(x=y_prob, color=cancellation_labels, nbins=50,
                   barmode="overlay",
                   color_discrete_sequence=[c[1][1], c[0][1]],
                   labels=dict(color=r'$\text{True Class Assignments}$',
                               x=r'$\text{Probability of Assigning Class }1$'),
                   title=r"$(3)\text{ Histogram of Class Assignment Probabilities}$",
                   height=350)

frames = [go.Frame(
    data=go.Scatter(x=[t / 10, t / 10], y=[0, 600], mode="lines",
                    line=dict(color="black"), showlegend=False), traces=[2])
          for t in range(11)]

fig.add_traces(frames[0]["data"][0])
    .update(frames=frames[1:])
    .update_layout(updatemenus=[
    dict(type="buttons", buttons=[AnimationButtons.play(frame_duration=1000),
                                  AnimationButtons.pause()])])

IndentationError: unexpected indent (1265416476.py, line 17)

In [ ]:
from sklearn.metrics import confusion_matrix

model = AgodaCancellationEstimator(0.3).fit(design_matrix, cancellation_labels)
y_prob = model.predict(test_set)
cm = confusion_matrix(test_set_labels, y_prob, labels=[0, 1])
print(cm)